# STL Model Classification Notes(week 01)

We classify STL files into **Flat Models** or **Free Models** using two main tests:  

---

## 1. Bounding Box Ratio (Thinness Test)

Think of putting your 3D model inside a cardboard box.  
The box has **Width (X), Height (Y), Thickness (Z)**.  

We calculate:

\[
\text{Ratio} = \frac{\text{Smallest Side}}{\text{Largest Side}}
\]

- If the **ratio is very small** → the model is **thin → Flat**   
- If the **ratio is close to 1** → the model is **volumetric → Free**   

**Examples:**  
- Plate (100 × 80 × 3) → ratio = 3/100 = **0.03** → **Flat**   
- Cube (50 × 50 × 50) → ratio = 50/50 = **1** → **Free**  

---

## 2. Surface Curvature (Flatness Test)

Every 3D model is made of **tiny flat triangles**.  
Each triangle has a **normal vector** → a little arrow showing its surface direction.  

- If the model is **flat**, most arrows point in the **same direction**.  
- If the model is **curved**, arrows point in **many directions**.  

We measure alignment with the average normal:

\[
\text{Aligned Ratio} = \frac{\text{Number of Normals Aligned}}{\text{Total Normals}}
\]

- If **Aligned Ratio ≥ 0.8 (80%)** → **Flat**   
- If **spread out** → **Free**  

**Examples:**  
- Plate → arrows almost parallel → **Flat**   
- Sphere → arrows spread in all directions → **Free** 

---

## Combined Rule

A model is classified as **Flat** if:  

\[
\text{Bounding Box Ratio} < \text{Threshold (e.g., 0.1)} \quad \text{AND} \quad \text{Aligned Ratio} > \text{Threshold (e.g., 0.8)}
\]

Otherwise, it is **Free**.  

 


In [1]:

import os
import trimesh
import numpy as np

def classify_models(folder_path, flat_ratio_threshold=0.1, normal_threshold=0.9, align_threshold=0.8):
    flat_models = []
    free_models = []

    for file in os.listdir(folder_path):
        if file.lower().endswith(".stl"):
            file_path = os.path.join(folder_path, file)
            try:
                mesh = trimesh.load(file_path)

                if not isinstance(mesh, trimesh.Trimesh):
                    continue  # skip multi-part meshes

                # --- Step 1: Bounding box ratio ---
                extents = mesh.bounding_box.extents
                min_dim = min(extents)
                max_dim = max(extents)
                ratio = min_dim / max_dim

                # --- Step 2: Surface curvature check ---
                normals = mesh.face_normals
                avg_normal = np.mean(normals, axis=0)
                avg_normal /= np.linalg.norm(avg_normal)  # normalize

                alignment = np.dot(normals, avg_normal)
                aligned_ratio = np.mean(alignment > normal_threshold)

                # --- Step 3: Combined classification ---
                if ratio < flat_ratio_threshold and aligned_ratio > align_threshold:
                    flat_models.append(file)
                else:
                    free_models.append(file)

            except Exception as e:
                print(f"Error loading {file}: {e}")

    return flat_models, free_models


# Example usage:
folder = r"C:\Users\shane\OneDrive\Desktop\3DP_Research\STL"
flat, free = classify_models(folder)

print("Flat Models:", flat)
print("Free Models:", free)


C:\Users\shane\AppData\Local\Temp\ipykernel_13160\766094055.py:28: RuntimeWarning: invalid value encountered in divide
  avg_normal /= np.linalg.norm(avg_normal)  # normalize


Flat Models: []
Free Models: ['1050061.stl', '1051177.stl', '1053375.stl', '1777452.stl', '60094.stl', '60099.stl', '60100.stl', '60101.stl', '60170.stl', '60172.stl', '60223.stl', '60246.stl', '60261.stl', '60262.stl', '60264.stl', '60265.stl', '60267.stl', '60268.stl', '60269.stl', '60270.stl', '60271.stl', '60273.stl', '60274.stl', '60275.stl', '60276.stl', '60277.stl', '60278.stl', '60304.stl', '60328.stl', '60364.stl', '60435.stl', '60512.stl', '60513.stl', '60514.stl', '60941.stl', '61082.stl', '61181.stl', '61434.stl', '61463.stl', '61585.stl', '61792.stl', '62286.stl', '62860.stl', '62987.stl', '62989.stl', '63245.stl', '63277.stl', '838514.stl', '838515.stl', '838517.stl']
